# 0. Imports

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import requests
import re
import time
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()


# 1. Leitura de bases

Os dados usados no desenvolvimento deste projeto surgiram a partir das bases presentes no link a seguir:

https://www.kaggle.com/datasets/natlee/myanimelist-comment-dataset

Osberve que a informação mais importante para nós são as que estão presentes na base `animeListGenres`

In [ ]:
nRowsRead = 'None' # specify 'None' if want to read whole file

with open('animeReviewsOrderByTime.csv', 'r', encoding='utf-8') as f:
    headers = f.readline().replace('"','').replace('\n','').split(',')
    print(headers)
    print('The number of column: ', len(headers))
    dataFormat = dict()
    for header in headers:
        dataFormat[header] = list()

    for idx, line in enumerate(tqdm(f.readlines(), desc='Now parsing... ')):
        
        if idx == 67:
            yee = line
        
        if line != '':
            line = line.replace('\n','')
            indices = [i for i, x in enumerate(line) if x == ',']
            idxStart = 0
            for i in range(len(headers)):
                if i < len(headers) - 1:
                    dataFormat[headers[i]].append(line[idxStart + 1:indices[i] - 1])
                    idxStart = indices[i] + 1
                elif i == len(headers) - 1:
                    dataFormat[headers[i]].append(line[idxStart + 1:-1])
                else:
                    break
        if nRowsRead is not None and nRowsRead == idx + 1:
            print('We read only', nRowsRead, 'lines.')
            break
aval = pd.DataFrame(dataFormat)
aval.head()

In [ ]:
#listinha serve para avaliar quais ids tivemoserro ao coletar a sinopse
listinha = []
def get_anime_synopsis(anime_id):
    time.sleep(1)
    url = f'https://myanimelist.net/anime/{anime_id}'
    response = requests.get(url)
    
    
    if response.status_code != 200:
        print(anime_id, "Parou aqui")
        listinha.append(anime_id)
        return "-" , "-"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    synopsis_element = soup.find("p", itemprop="description")
    synopsis = synopsis_element.text.strip() if synopsis_element else 'Synopsis not found.'
    for image_tag in soup.find_all("img",itemprop="image"):
        image_link = image_tag['data-src']

    return synopsis , image_link

Como não temos a sinopse dos anims vamos coleta-las diretod a fonte, coletando de 100 em 100 e pausando por 10 min

In [ ]:
# Carregando os dados
animes = pd.read_excel("Base_animes_tratados.xlsx")

In [ ]:
#Coletando sinopse dos animes
for i in tqdm(len(animes)):
    try:
        start = i * 100
        end = (i + 1) * 100 - 1
        result=pd.DataFrame()
        result[['synopsis', 'IMG']] = animes.loc[start:end, 'workId'].apply(get_anime_synopsis).apply(pd.Series)
        animes.loc[start:end, ['synopsis', 'IMG']] = result
    except Exception as e:
        print(f"An error occurred: {e}. Ignoring and moving on to the next chunk.")
        continue
    animes.to_excel("Nase_animes_tratados.xlsx", index = False)
    time.sleep(600)  # Sleep for 600 seconds (10 minutes) after each chunk

# 2. Tratamento dos dados

In [ ]:
animes = animes[(animes["IMG"] !="-") & (animes["synopsis"] !="-")]
# Removendo duplicatas
animes.drop_duplicates(inplace=True)
aval.drop_duplicates(inplace=True)

# Tratando valores faltantes
animes.fillna("", inplace=True)
aval.fillna("", inplace=True)

# Instanciando o stemmer and the lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


In [ ]:
import nltk
from nltk.corpus import stopwords

animes.drop_duplicates(inplace=True)
aval.drop_duplicates(inplace=True)

# Tratando valores faltantes
animes.fillna("", inplace=True)
aval.fillna("", inplace=True)

# Instanciando o stemmer and the lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Baixe as stopwords em português (se ainda não tiver baixado)
nltk.download('stopwords')

# Carregue as stopwords em português
remover_palavras = stopwords.words('english')#stopwords.words('portuguese')

from nltk.stem import RSLPStemmer
import spacy

# Baixe os recursos necessários (se ainda não tiver baixado)
nltk.download('rslp')
nltk.download('wordnet')

# Crie uma instância do stemmer RSLP
stemmer = RSLPStemmer()

# Crie uma instância do lematizador para o português
nlp = spacy.load('en_core_web_sm')#spacy.load('pt_core_news_sm')
# Function to remove HTML tags
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to preprocess the text
def preprocess(text):
    try:
        # Removing HTML tags
        text = remove_html(text)
    except:
        pass
        # Removing URLs
    try:
        text = remove_urls(text)
    except:
        pass
        # Removing emojis
    try:
        text = remove_emojis(text)
    except:
        pass
        # Converting to lowercase
    try:
        text = text.lower()
    except:
        pass
        # Removing special characters
    try:
        text = re.sub(r'\W', ' ', text)
    except:
        pass
        # Removing extra spaces
    try:
        text = re.sub(r'\s+', ' ', text, flags=re.I)
    except:
        pass
        # Removing stopwords, applying lemmatization and then stemming
    try:
        #text = ' '.join([stemmer.stem(lemmatizer.lemmatize(word)) for word in text.split() if word not in remover_palavras])
        text = ' '.join([stemmer.stem(token.lemma_) for token in nlp(text) if token.lemma_ not in remover_palavras])
    except:
        pass
    
    return text

In [ ]:
# Function to remove HTML tags
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Function to remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to preprocess the text
def preprocess(text):
    try:
        # Removing HTML tags
        text = remove_html(text)
    except:
        pass
        # Removing URLs
    try:
        text = remove_urls(text)
    except:
        pass
        # Removing emojis
    try:
        text = remove_emojis(text)
    except:
        pass
        # Converting to lowercase
    try:
        text = text.lower()
    except:
        pass
        # Removing special characters
    try:
        text = re.sub(r'\W', ' ', text)
    except:
        pass
        # Removing extra spaces
    try:
        text = re.sub(r'\s+', ' ', text, flags=re.I)
    except:
        pass
        # Removing stopwords, applying lemmatization and then stemming
    try:
        #text = ' '.join([stemmer.stem(lemmatizer.lemmatize(word)) for word in text.split() if word not in remover_palavras])
        text = ' '.join([stemmer.stem(token.lemma_) for token in nlp(text) if token.lemma_ not in remover_palavras])
    except:
        pass
    
    return text


In [ ]:
# Applying the preprocess function to the synopsis and user comments
animes['synopsis tratada'] = animes['synopsis'].progress_apply(preprocess)


In [ ]:
aval['review tratada'] = aval['review'].progress_apply(preprocess)

In [ ]:
animes['synopsis tratada final'] = animes['synopsis tratada'].progress_apply(preprocess)

In [ ]:
animes.reset_index(inplace= True, drop = True)

In [ ]:
#Existem alguns nomes em branco
animes["engName"].value_counts()

In [ ]:
#atualizando nome dos animes
def atualizar_nome(row):
    if row["engName"] != "\\N":
        return row["engName"]
    elif row["synonymsName"] != "\\N":
        return row["synonymsName"]
    else:
        return row["jpName"]
animes["name"] = animes.apply(lambda row: atualizar_nome(row), axis = 1)


In [ ]:
#Note que reduzimos os sem nome para 13, vamos mantelos
animes["name"].value_counts()

In [ ]:
animes = animes[-animes["synopsis tratada final"].isna()].reset_index(drop=True)
animes.to_excel("base_tratada_final_animes.xlsx",index = False)

In [ ]:
aval.to_csv("reviews_tratada.csv", index = False)

# 3. Similadirade

In [ ]:
aval = pd.read_csv("reviews_tratada.csv")
animes = pd.read_excel("base_tratada_final_animes.xlsx")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Vectorize the preprocessed anime descriptions
vectorizer = TfidfVectorizer()
vectorized_descriptions = vectorizer.fit_transform(animes['synopsis tratada final'])

# Calculate similarity scores
similarity_matrix = cosine_similarity(vectorized_descriptions)

# Function to retrieve top similar animes for a given anime
def get_top_similar_animes(anime_index, similarity_matrix, top_k=5):
    anime_scores = similarity_matrix[anime_index]  # Get similarity scores for the given anime
    top_indices = anime_scores.argsort()[-top_k-1:-1][::-1]  # Get top similar anime indices (excluding the given anime itself)
    return animes.iloc[top_indices]['workId'].values.tolist()




In [ ]:
np.save("vector", similarity_matrix)   

In [ ]:
animes[animes["name"] == "Wolf Children"]

In [ ]:
similar_animes = get_top_similar_animes(34, similarity_matrix, top_k=5)


In [ ]:
genero = animes.loc[34, "genres"]

In [ ]:
genero

In [ ]:
base_auxiliar = animes[animes["workId"].isin(similar_animes)]
base_auxiliar["ordem"] = pd.Categorical(base_auxiliar["workId"], categories=similar_animes, ordered=True)
base_auxiliar = base_auxiliar.sort_values("ordem").reset_index()
base_auxiliar.head()

In [ ]:
# Calculate Fuzzy Ratio for each row
base_auxiliar['fuzzy_ratio'] = base_auxiliar['genres'].apply(lambda x: fuzz.ratio(x, genero))



def classifica(row):
    if row['fuzzy_ratio'] > 0.8:
        return 4
    elif row['fuzzy_ratio'] > 0.6:
        return 3
    elif row['fuzzy_ratio'] > 0.4:
        return 2
    elif row['fuzzy_ratio'] > 0.2:
        return 1
    elif row['fuzzy_ratio'] > 0:
        return 0

In [ ]:
base_auxiliar

In [ ]:
base_auxiliar['fuzzy_category'] = base_auxiliar.apply(lambda row: classifica(row) ,axis = 1)



In [ ]:
base_auxiliar

In [ ]:
# Sort by Fuzzy Ratio categories, then by original order
base_auxiliar.sort_values('fuzzy_category', ascending=False, kind='mergesort', inplace=True)

# You can drop the helper columns if not needed
base_auxiliar.drop(['fuzzy_ratio', 'fuzzy_category'], axis=1, inplace=True)
base_auxiliar.reset_index(inplace = True , drop = True)